In [78]:
import cv2
import numpy as np
import imutils

def get_max(l):
    w = []
    h = []
    for i in range(len(l)):
        w.append(l[i][0])
        h.append(l[i][1])
    return (max(w),max(h))

def rotate_image(image, angle):
  center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
  res = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return res


image = cv2.imread('./scan.jpg')

img = cv2.resize(image, (500,600))
orig = image.copy()
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(gray, 200, 255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
edge = cv2.Canny(th,70,200)

cnts,_ = cv2.findContours(edge.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break

l = []
for i in range(len(screenCnt)):
    m = screenCnt[i][0][0]
    n = screenCnt[i][0][1]
    l.append([m,n])

width,height = get_max(l) 

dst = np.array([[0, 0],[width, 0],[width, height],[0, height]], dtype = "float32")
src = np.array(l,dtype = "float32")

M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(img, M, (width, height))
warped = cv2.flip(warped,1)
final = rotate_image(warped,90)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
final = cv2.filter2D(final, -1, kernel)

cv2.drawContours(img,[c],-1,(255,255,0))
cv2.imshow('Contour',img)
cv2.imshow('Warped',final)
cv2.waitKey()
cv2.destroyAllWindows()